In [1]:
import pandas as pd
import numpy as np

from collections import Counter
from itertools import permutations

import networkx as nx

import os

In [2]:
def print_edges(edges):
    print(','.join([str(e) for e in edges]))

# Считываем граф

Oставляем только большую компоненту связности

In [3]:
G = nx.DiGraph()

with open("assembly/K127/prelim_graph.gfa") as f:
    for line in f:
        line = line.split()
        line_type = line[0]
        
        # S 238024 ACCAATTAT KC:i:37210
        if line_type == "S":
            v_name = int(line[1])
            v_length = len(line[2])
            G.add_node(v_name, length=v_length)
        
        # L 238322 + 19590 - 55M
        if line_type == "L":
            v1 = int(line[1])
            v2 = int(line[3])
            G.add_edge(v1, v2)
            
print('Number of components:', nx.number_weakly_connected_components(G))      
            
# remain only largest component
new_G = nx.DiGraph()
for g in nx.weakly_connected_component_subgraphs(G):
    print(g.number_of_nodes())
    if new_G.number_of_nodes() < g.number_of_nodes():
        new_G = g.copy()
G = new_G.copy()

Number of components: 213
3111
2
1
1
2
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
1
1
1
1
1
1
1
1
1
1
1
2
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


# Табличка с референсами

Считываем файл ответа, как он есть

In [4]:
df_ref = pd.read_csv("refs/refs_edges.txt", header=None, names=["e"])

df_ref = df_ref["e"].str.split('\t', 1, expand=True)
df_ref.columns = ["e_id", "strains"]
df_ref = df_ref.set_index("e_id")
df_ref.index = df_ref.index.astype("int")
df_ref.head()

,strains
e_id,
3051154,s9_8\ts6_1
2899616,s6_1
3094698,s9_8
2935900,s6_1
2652616,s6_1


Оставляем только ребра из большой компоненты:

In [5]:
df_ref = df_ref.loc[list(G.nodes)]

Выводим рёбра, которые никому не принадлежат, на экран, и выкидываем их из таблицы:

In [6]:
nobody_edges = df_ref[df_ref["strains"].isnull()].index
print("{n} edges are Nobody".format(n=len(nobody_edges)), '\n')
print_edges(nobody_edges)

df_ref = df_ref.loc[set(df_ref.index) - set(nobody_edges)]

152 edges are Nobody 

729174,3088472,3096792,3031324,860472,3088702,3088748,573858,697314,3007572,3097690,549984,492682,754962,427370,345464,296672,3098422,3098516,2803630,3008452,3098612,3098614,3098630,2992144,469208,3098894,3090860,3099142,35410,3099352,60138,3099394,3099422,3099442,52020,3099466,3099472,3099478,3099530,3099572,2878418,2968584,3099750,3099812,584978,2977164,93618,822772,3084050,3970,3084272,3084280,3084288,3084294,2699274,413714,2633754,3100712,2977882,594124,3100972,2634046,3101016,2879852,528752,3084726,291264,2634264,2634292,3027512,242300,848530,406166,3085038,2986754,472022,504790,2987088,2634836,2962672,2635004,2815234,3093794,2635068,3085692,3102122,2635270,2668084,513620,3102368,38598,2766560,3102448,2635526,137132,2635698,3102662,432070,407536,407538,2938888,3102740,2635832,3102828,47402,3103042,3094850,3103052,825696,3103086,3103108,2636170,358814,2636198,547280,2636246,3103196,498146,367082,276994,621094,3103380,3095228,3095258,2636516,3021554,3103480,31

Сплитим список референсов:

In [7]:
df_ref["strains"] = df_ref["strains"].str.split('\t')
df_ref["strains"] = df_ref["strains"].apply(lambda x: [s.rpartition('_')[0] for s in x])
df_ref["strains"] = df_ref["strains"].apply(Counter)
df_ref.head()

,strains
e_id,
2719746,{'s9': 2}
3096592,{'s6': 1}
2981908,{'s6': 1}
3096602,{'s9': 1}
3096610,"{'s9': 1, 's6': 1}"


Считаем копийность каждого ребра:

In [8]:
df_ref["single_copy"] = df_ref["strains"].apply(lambda x: x.most_common(1)[0][1] == 1)
df_ref.head()

,strains,single_copy
e_id,,
2719746,{'s9': 2},False
3096592,{'s6': 1},True
2981908,{'s6': 1},True
3096602,{'s9': 1},True
3096610,"{'s9': 1, 's6': 1}",True


Считываем длину рёбер:

In [9]:
df_length = pd.read_csv("edges_lengths.tsv", sep='\t', header=None, names=["length"])
df_length = df_length.astype("int")

df_ref = df_ref.join(df_length, how='inner')
df_ref.head()

,strains,single_copy,length
2719746,{'s9': 2},False,140
3096592,{'s6': 1},True,1674
2981908,{'s6': 1},True,305
3096602,{'s9': 1},True,2779
3096610,"{'s9': 1, 's6': 1}",True,8878


# Считываем профили

In [10]:
ref_profile = pd.read_csv("refs/profile.csv", header=None, index_col=0)
for i in range(1, 11):
    ref_profile[i] = ref_profile[i] / ref_profile[i].sum()
ref_profile

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
9,0.41,0.83,0.98,0.74,0.09,0.82,0.49,0.99,0.37,0.99
6,0.59,0.17,0.02,0.26,0.91,0.18,0.51,0.01,0.63,0.01


In [11]:
desman_profile = pd.read_csv("desman_results/%s_0/desman_freqs.csv" % len(ref_profile),
                             header=None, index_col=0, dtype=float)
desman_profile.index = desman_profile.index.astype(int)
desman_profile

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
0,0.435,0.841,0.984,0.753,0.127,0.838,0.508,0.991,0.396,0.991
1,0.565,0.159,0.016,0.247,0.873,0.162,0.492,0.009,0.604,0.009


Ищем соответствие между профилями:

In [12]:
ref_freqs = ref_profile.as_matrix()
ans_error = float("Inf")
ans_permut = None
for cur_permut in permutations(desman_profile.index):
    desman_freqs = desman_profile.loc[cur_permut, :].as_matrix()
    #print(cur_error, cur_permut)
    cur_error = ((ref_freqs - desman_freqs) ** 2).sum()
    if cur_error < ans_error:
        ans_error = cur_error
        ans_permut = cur_permut
print("Error:", ans_error)

Error: 0.007252


In [13]:
def invert_permutation(permutation):
    return [i for i, j in sorted(enumerate(permutation), key=lambda x: x[1])]

In [15]:
strains = list('s' + ref_profile.iloc[invert_permutation(ans_permut), :].index.astype(str))
strains

['s9', 's6']

# Табличка ответов DESMAN

In [16]:
df_desman = pd.read_csv("desman_results/%s_0/gene_assignment_etaS_df.csv" % len(strains), skiprows=1,
                  names=["e_id"] + strains)
df_desman['e_id'] = df_desman['e_id'].str[1:].astype("int")
df_desman = df_desman.set_index('e_id')
df_desman[strains] = df_desman[strains].astype('int')

#df_desman = df_desman.join(df_length, how='inner')

df_desman = df_desman.loc[set(df_ref.index) - set(nobody_edges)]
df_desman.head()

,s9,s6
e_id,,
2719746,1,0
3096592,0,1
2981908,0,1
3096602,1,0
3096610,1,0


In [17]:
for cur_s in strains:
    df_ref[cur_s] = df_ref['strains'].apply(lambda x: int(cur_s in x))
    
df_ref.head()

,strains,single_copy,length,s9,s6
2719746,{'s9': 2},False,140,1,0
3096592,{'s6': 1},True,1674,0,1
2981908,{'s6': 1},True,305,0,1
3096602,{'s9': 1},True,2779,1,0
3096610,"{'s9': 1, 's6': 1}",True,8878,1,1


# Точность DESMAN

In [18]:
df_ref.sort_index(inplace=True)
df_desman.sort_index(inplace=True)

In [19]:
right_answers = (df_ref[strains] == df_desman[strains]).sum(axis=1) == len(strains)
print("Accuracy on all edges: %.2f" % (right_answers.sum() / len(df_ref)))

Accuracy on all edges: 0.68


In [20]:
long_edges = df_ref['length'] > 500
print("Percent of long edges: %.2f" % (long_edges.sum() / len(df_ref)))
print("Accuracy on long edges: %.2f" % ((right_answers & long_edges).sum() / long_edges.sum()))

Percent of long edges: 0.82
Accuracy on long edges: 0.69


# Раскрашиваем граф для конкретного штамма

In [21]:
if not os.path.exists("bandage_colors"):
    os.makedirs("bandage_colors")


for cur_s in strains:
    print('\n\n', "_________________________", cur_s, '\n')
    
    selected_nodes = df_ref[df_ref[cur_s] == 1].index
    G_sub = G.subgraph(selected_nodes)
    print("components in reference subgraph:", nx.number_weakly_connected_components(G_sub))

    selected_nodes = df_desman[df_desman[cur_s] == 1].index
    G_sub = G.subgraph(selected_nodes)
    print("components in   DESMAN  subgraph:", nx.number_weakly_connected_components(G_sub))

    df_ref['color'] = "#b0b0b0"  # grey


    long = df_ref['length'] >= 500
    single = df_ref['single_copy']
    real_true = df_ref[cur_s] == 1
    desman_true = df_desman[cur_s] == 1

    df_ref.loc[~long & real_true, 'color'] = 'Brown'

    df_ref.loc[long & single  &  real_true  &  desman_true, 'color'] = 'Lime'
    df_ref.loc[long & ~single  &  real_true  &  desman_true, 'color'] = 'Green'

    df_ref.loc[long &  single  &  real_true  & ~desman_true, 'color'] = 'Teal'
    df_ref.loc[long & ~single  &  real_true  & ~desman_true, 'color'] = 'Navy'

    df_ref.loc[long &  single  & ~real_true  &  desman_true, 'color'] = 'Yellow'
    df_ref.loc[long & ~single  & ~real_true  &  desman_true, 'color'] = 'Orange'


    df_ref['strains_print'] = df_ref['strains'].apply(lambda x: ", ".join('{}({})'.format(k, v) for k, v in x.items()))
    df_ref['strains_print'] = df_ref['strains_print'].apply(lambda x: x.replace('(1)', ''))

    df_ref[['strains_print', 'color']].to_csv("bandage_colors/{}.csv".format(cur_s), index_label='name')
    
    print("\nlong FN")
    print_edges(df_ref[long & real_true & ~desman_true].index)
    
    print("\nlong FP")
    print_edges(df_ref[long & ~real_true & desman_true].index)



 _________________________ s9 

components in reference subgraph: 1
components in   DESMAN  subgraph: 22

long FN
3104004

long FP
2636772,2702468,2713804,2720574,2772264,2774372,2793100,2824318,2831124,2859096,2867600,2883122,2892396,2893568,2899676,2902722,2916484,2922788,2928688,2932392,2932886,2934428,2935938,2938262,2942248,2957606,2958440,2958574,2958918,2959666,2959850,2960582,2960832,2961778,2963054,2963380,2965194,2966204,2966448,2968910,2968944,2969870,2970378,2971380,2971916,2971960,2972212,2973592,2974534,2974998,2975152,2976938,2978002,2978272,2978986,2979040,2979130,2979952,2980184,2980312,2980930,2981574,2982708,2983094,2984292,2984914,2985162,2985766,2987998,2988864,2988942,2989212,2989532,2989642,2989646,2989870,2992374,2995236,2996364,3004272,3004868,3005364,3007012,3013726,3084856,3085372,3086446,3086732,3087016,3087936,3088714,3094106,3094388,3095060,3095446,3095648,3095700,3095748,3096308,3096436,3096446,3096752,3096972,3097144,3097682,3097708,3097896,3098010,309